# OpenVINO APIの基礎的な使い方を学ぶ  

ここではOpenVINO APIを使用して推論を実行するのに最低限必要なAPIとその使用順番などについて学びます。  
ダミーデータを用いて推論を実行するので、推論結果そのものに意味はありませんが、OpenVINOを使った推論プログラムの基本骨格を理解してください。

## 必要なライブラリを読み込み  

ここでは`numpy`と`openvino`を読み込んでいます。

In [ ]:
import numpy as np

import openvino as ov

## IRモデルファイルの読み込み
本来、IRモデルファイルの読み込みに`read_model()`、読み込んだIRモデルを指定デバイス用にコンパイル(最適化)するのに`compile_model()`の２つのAPIを使用するが、`compile_model()`APIの引数にファイル名を指定することでファイルの読み込みからデバイス向け最適化までを１APIで実行可能。
```python
ov_core = ov.Core()
ov_model = ov_core.read_model('resnet50.xml')
compiled_model = ov.compile_model(ov_model, 'CPU')
```
と、
```python
compiled_model = ov.compile_model('resnet50.xml', device_name='CPU')
```
は等価です。

In [ ]:
compiled_model = ov.compile_model('resnet50.xml', device_name='CPU')

## 推論に使用する入力データを準備

ここではダミーデータを用意しています。このチュートリアルの推論結果には意味がありません。

In [ ]:
dummy_input = np.zeros((1, 3, 224, 224))
print(dummy_input.shape)

## 推論実行  
ここでは`infer_new_request()` APIを使用して推論します。OpenVINOで推論をする場合に一番手っ取り早い方法です。  
モデルの入力が一つしかない場合は、下記セルのように推論データを引数として渡すだけでよいのですが、モデルの入力が複数ある場合は辞書(dict)形式で与えます。(`model.infer_new_request({'input1':data1, 'input2':data2}`のような感じ)  

`%%time`はJupyter notebookの機能です。セル全体の実行時間を計測します。

In [ ]:
%%time
result = compiled_model.infer_new_request(dummy_input)

## 推論結果へのアクセス方法

In [ ]:
print(compiled_model.outputs)                       # outputsを表示させると出力ノード名とそのプロパティ(シェイプなど)を確認できます
print(compiled_model.outputs[0].get_any_name())     # compiled_model.outputsで出力ノードのリストを取得できます。`get_any_name()` APIで出力ノード名を得ることができます。
print(result['x.45'].shape)                         # 推論結果はノード名でアクセス可能です
print(result[0].shape)                              # インデックス番号で推論結果にアクセスすることもできます

## 推論結果の表示  

推論結果を表示してみます。  
|パラメータ|説明|
|---|---|
|`names[]`|推論結果出力ノード名のリストです。１つの出力が複数の名前を持っている場合もあります（一つも持っていない場合もあります。その場合はインデックス番号でアクセスします）|
|`shape[]`|推論結果のシェイプです。この場合`[?,1000]`なので、推論結果には1,000個のfp32の数値データが出てくることがわかります。|

In [ ]:
print(result)

## 推論結果へのアクセス  

下記のセルのように、推論結果へは出力ノード名でもインデックス番号でもアクセスが可能です。  
どちらの手段でも同じ結果が出ることを確認してください。

In [ ]:
pred = result['x.45']     # ノード名でアクセス
print(pred.shape)
print(pred[0, :50])

pred = result[0]          # インデックス番号でアクセス
print(pred.shape)
print(pred[0, :50])

## おわり  

これでOpenVINO APIを使用して最低限の推論プログラムを実行する手順を学びました。次からは実際に意味のある推論プログラムを実行しながら、OpenVINOのより詳細な使用方法を学びましょう。